In [1]:
from music21 import *

In [4]:
"""
Parse the Teslim and extract information relevant to inferring the makam.
- input: path to the musicxml for a sarki
- return: a dictionary of the following format:
    {
    "start_note": first note of the Teslim (music21 Note object),
    "end_note": last note of the Teslim (music21 Note),
    "range": (lowest note in the Teslim, highest note of the Teslim) (pair of music21 Notes),
    "accidentals": list of notes with accidentals in the order that they appear in the Teslim (list of music21 Notes)
    }
"""
filename = "../With Key Signatures/CT1.musicxml"

def teslim_finder(filename):
    MI = musicxml.xmlToM21.MusicXMLImporter()
    score = MI.scoreFromFile(filename)
    start_measure_num = None
    end_measure_num = None

    for element in score.recurse().getElementsByClass(expressions.TextExpression):
        if element.content.lower() == 'teslim':
            start_measure_num =  element.getContextByClass('Measure').number
        if element.content.lower() == 'hane 2':
            end_measure_num = element.getContextByClass('Measure').number - 1
            break

    teslim = stream.Score()

    # Iterate through the measures in the original score and add them to the new score
    for measure in score.measures(start_measure_num, end_measure_num):
        teslim.insert(measure.offset, measure)

    return teslim

teslim_finder(filename)

<music21.stream.Score 0x1c0bdd87990>


<music21.stream.Score 0x1c0bdd87990>